In [1]:
#Importing other jupyter notebook (calculations)
import Pkg
Pkg.add("NBInclude")
Pkg.add("IJulia")
using NBInclude, IJulia

@nbinclude("plumeSSMEJP.ipynb")
IJulia.installkernel("Julia nodeps", "--depwarn=no")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.t

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


┌ Info: Installing Julia nodeps kernelspec in /home/chinahg/.local/share/jupyter/kernels/julia-nodeps-1.5
└ @ IJulia /home/chinahg/.julia/packages/IJulia/e8kqU/deps/kspec.jl:94


"/home/chinahg/.local/share/jupyter/kernels/julia-nodeps-1.5"

In [3]:
#PLOTTING ONLY IN THIS NOTEBOOK
h = Int.(LinRange(16000, 40000, space)) #make an array and put in loop for all alts
#h = [24000]
for m = 1:length(h)
    h_string = string(h[m])
    print(h_string)
    print("\n")

    T = T_save[m,:,:]
    u = u_save[m,:,:]
    χ = χ_save[m,:,:,:]

    #REGRID SOLUTION
    xx, yy, u_g, T_g, χ_gH2O =  regrid_solution(x, y, u, T, χ[:,:,6], 0.01);
    xx, yy, u_g, T_g, χ_gNO2 =  regrid_solution(x,y, u, T, χ[:,:,37], 0.01);
    xx, yy, u_g, T_g, χ_gN2O =  regrid_solution(x, y, u, T, χ[:,:,38], 0.01);
    xx, yy, u_g, T_g, χ_gNO =  regrid_solution(x, y, u, T, χ[:,:,36], 0.01);

    #PLOT 2D MAPS AND SAVE
    plot_H2O = plot_heatmap(xx, yy, χ_gH2O, "Axial distance, m", "Radial distance, m","Concentration", :magma)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_H2O.png")

    #CALCULATE NO EI FOR ALTITUDE
    # Sum of all NOx species as a function of x
    sumNO = zeros(n);
    sumNO2 = zeros(n);
    sumN2O = zeros(n);
    i = 1
    j = 1

    for i in 1:n-1
        if xx[i] < 0 || xx[i] == NaN
            xx[i] = (xx[i-1] + xx[i+1])/2
        end
        for j in 1:50
            #println(j," ", i)
            #println(χ_gNO[j,i])
            if χ_gNO[j,i] < 0 || χ_gNO[j,i]== NaN 
                println(j)
                println(i)
                χ_gNO[j,i] = 0
            
            elseif χ_gNO2[j,i] < 0 || χ_gNO2[j,i]== NaN
                println(j)
                println(i)
                χ_gNO2[j,i] = 0
            
            elseif χ_gN2O[j,i] < 0 || χ_gN2O[j,i]== NaN
                println(j)
                println(i)
                χ_gN2O[j,i] = 0
            end
        end 
    end
    #print(χ_gNO[:,20])
    i = 1
    for i in 1:n
        sumNO[i] = sum(χ_gNO[:,i])
        sumNO2[i] = sum(χ_gNO2[:,i])
        sumN2O[i] = sum(χ_gN2O[:,i])
    end

    sumNOx = (sumNO+sumNO2+sumN2O)/(50) #total ppm of NOx

    #ppm to kg
    #Volume for each "ring" so we can turn ppm to kg :)
    f = 1 #altitude index
    volume = zeros(50,27)
    NOx_mass = zeros(length(h),27)
    for i = 2:27, j = 2:50
        volume[j,i] = 0.27*3.1415*(xx[i]-xx[i-1])*(yy[j]^2 - yy[j-1]^2)
        NOx_mass[f,i] += (χ_gNO[j,i]+χ_gNO2[j,i]+χ_gN2O[j,i])*volume[j,i]
    end

    plotNOxMass = plot(xx[1:27], NOx_mass[f,:], ylabel = "Mass NOx", xlabel = "Distance from nozzle exit [m]", label = "NOx [kg]", lw = 3)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/NOxMass.png")

    mdot_f = 67.35 #kg/s
    MW_NO2 = 46
    MW_NO = 30
    NOxEI = (1000 .*NOx_mass[f,:] ./(100*(mdot_f*1.333)))*MW_NO2/MW_NO
    plotNOxEI = plot(xx[1:27], NOxEI, ylabel = "EI NOx", xlabel = "Distance from nozzle exit [m]", label = "NOx EI [g/kg]", lw = 3)
    savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/NOxEI.png")

    #NOx mass for each altitude
    NOx_mass_alt = zeros(length(h))
    NOx_mass_alt[f] = sum(NOx_mass[f,:])*100; #assuming per 10m, times 100 for 1000m change in altitude
end

16000


LoadError: [91mUndefVarError: χ not defined[39m